<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning

The notebook will train, create and deploy a Credit Risk model. It will then configure OpenScale to monitor drift in data and accuracy by injecting sample payloads for viewing in the OpenScale Insights dashboard.

### Contents

- [1. Setup](#setup)
- [2. Model building and deployment](#model)
- [3. OpenScale configuration](#openscale)
- [4. Generate drift model](#driftmodel)
- [5. Submit payload](#payload)
- [6. Enable drift monitoring](#monitor)
- [7. Run drift monitor](# )

# 1.0 Setup <a name="setup"></a>

## 1.1 Package installation

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!rm -rf /home/spark/shared/user-libs/python3.6*
!pip install --upgrade opt-einsum==2.3.2 --no-cache | tail -n 1
!pip install --upgrade typing-extensions==3.6.2.1 --no-cache | tail -n 1
!pip install --upgrade jupyter==1 --no-cache | tail -n 1
!pip install --upgrade tensorboard==1.15.0 | tail -n 1
!pip install --upgrade ibm-ai-openscale==2.2.1 --no-cache | tail -n 1
!pip install --upgrade JPype1-py3 | tail -n 1
!pip install --upgrade watson-machine-learning-client-V4==1.0.93 | tail -n 1
!pip install --upgrade numpy==1.18.3 --no-cache | tail -n 1
!pip install --upgrade SciPy==1.4.1 --no-cache | tail -n 1
!pip install --upgrade pyspark==2.3 | tail -n 1
!pip install --upgrade scikit-learn==0.20.3 | tail -n 1
!pip install --upgrade pandas==0.24.2 | tail -n 1
!pip install --upgrade ibm-wos-utils>=1.2.1

ERROR: Could not install packages due to an EnvironmentError: [Errno 30] Read-only file system: 'INSTALLER'

    Uninstalling ibm-ai-openscale-2.1.21:
Requirement already up-to-date: JPype1-py3 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (0.5.5.4)
Requirement already up-to-date: numpy==1.18.3 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (1.18.3)


### Action: restart the kernel!

## 1.2 Configure credentials

- WOS_CREDENTIALS (ICP)
- WML_CREDENTIALS (ICP)
- DATABASE_CREDENTIALS (DB2 on ICP)
- SCHEMA_NAME

The url for `WOS_CREDENTIALS` is the url of the CP4D cluster, i.e. `https://zen-cpd-zen.apps.com`.

In [3]:
WOS_CREDENTIALS = {
    "url": "********",
    "username": "********",
    "password": "********"
}

In [4]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='3.0.0'

Provide `DATABASE_CREDENTIALS`. Watson OpenScale uses a database to store payload logs and calculated metrics. If an OpenScale datamart exists in Db2, the existing datamart will be used and no data will be overwritten. Details in the cell below is removed as it contains password. 

In [5]:
DATABASE_CREDENTIALS = {

}

Provide SCHEMA_NAME. Details in the cell below is removedDetails in the cell below is removed

In [6]:
SCHEMA_NAME = ''

Provide a custom name to be concatenated to model name, deployment name and open scale monitor. Sample value for CUSTOM_NAME could be ```CUSTOM_NAME = 'SAMAYA_OPENSCALE_3.0'```

In [7]:
CUSTOM_NAME = 'SAMAYA-DRIFT'

# 2.0 Model building and deployment <a name="model"></a>

In this section you will learn how to train Spark MLLib model and next deploy it as web-service using Watson Machine Learning service.

## 2.1 Load the training data

In [8]:
import pandas as pd
!rm -rf german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/cpd-intelligent-loan-agent-assets/master/data/german_credit_data_biased_training.csv  -O german_credit_data_biased_training.csv
    
!ls -lh german_credit_data_biased_training.csv

data_df = pd.read_csv('german_credit_data_biased_training.csv', sep=",", header=0)


--2020-06-16 05:31:12--  https://raw.githubusercontent.com/IBM/cpd-intelligent-loan-agent-assets/master/data/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

100%[======================================>] 689,622     --.-K/s   in 0.02s   

2020-06-16 05:31:13 (27.9 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]

-rw-r-----. 1 wsuser watsonstudio 674K Jun 16 05:31 german_credit_data_biased_training.csv


In [9]:
data_df.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


In [10]:
from pyspark.sql import SparkSession
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

Row(CheckingStatus='0_to_200', LoanDuration=31, CreditHistory='credits_paid_to_date', LoanPurpose='other', LoanAmount=1889, ExistingSavings='100_to_500', EmploymentDuration='less_1', InstallmentPercent=3, Sex='female', OthersOnLoan='none', CurrentResidenceDuration=3, OwnsProperty='savings_insurance', Age=32, InstallmentPlans='none', Housing='own', ExistingCreditsCount=1, Job='skilled', Dependents=1, Telephone='none', ForeignWorker='yes', Risk='No Risk')

## 2.2 Explore data

In [11]:
df_data.printSchema()

root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = true)



In [12]:
print("Number of records: " + str(df_data.count()))

Number of records: 5000


## 2.3 Create a model

Choose a unique name (i.e. your name or initials) and a date or date-time for `MODEL_NAME` and `DEPLOYMENT_NAME`

In [13]:
MODEL_NAME = CUSTOM_NAME + "_MODEL"
DEPLOYMENT_NAME = CUSTOM_NAME + "_DEPLOYMENT"

In [14]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

Number of records for training: 4016
Number of records for evaluation: 984
root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = 

The code below creates a Random Forest Classifier with Spark, setting up string indexers for the categorical features and the label column. Finally, this notebook creates a pipeline including the indexers and the model, and does an initial Area Under ROC evaluation of the model.

In [15]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model

si_CheckingStatus = StringIndexer(inputCol = 'CheckingStatus', outputCol = 'CheckingStatus_IX')
si_CreditHistory = StringIndexer(inputCol = 'CreditHistory', outputCol = 'CreditHistory_IX')
si_LoanPurpose = StringIndexer(inputCol = 'LoanPurpose', outputCol = 'LoanPurpose_IX')
si_ExistingSavings = StringIndexer(inputCol = 'ExistingSavings', outputCol = 'ExistingSavings_IX')
si_EmploymentDuration = StringIndexer(inputCol = 'EmploymentDuration', outputCol = 'EmploymentDuration_IX')
si_Sex = StringIndexer(inputCol = 'Sex', outputCol = 'Sex_IX')
si_OthersOnLoan = StringIndexer(inputCol = 'OthersOnLoan', outputCol = 'OthersOnLoan_IX')
si_OwnsProperty = StringIndexer(inputCol = 'OwnsProperty', outputCol = 'OwnsProperty_IX')
si_InstallmentPlans = StringIndexer(inputCol = 'InstallmentPlans', outputCol = 'InstallmentPlans_IX')
si_Housing = StringIndexer(inputCol = 'Housing', outputCol = 'Housing_IX')
si_Job = StringIndexer(inputCol = 'Job', outputCol = 'Job_IX')
si_Telephone = StringIndexer(inputCol = 'Telephone', outputCol = 'Telephone_IX')
si_ForeignWorker = StringIndexer(inputCol = 'ForeignWorker', outputCol = 'ForeignWorker_IX')

In [16]:
si_Label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [17]:
va_features = VectorAssembler(inputCols=["CheckingStatus_IX", "CreditHistory_IX", "LoanPurpose_IX", "ExistingSavings_IX", "EmploymentDuration_IX", "Sex_IX", \
                                         "OthersOnLoan_IX", "OwnsProperty_IX", "InstallmentPlans_IX", "Housing_IX", "Job_IX", "Telephone_IX", "ForeignWorker_IX", \
                                         "LoanDuration", "LoanAmount", "InstallmentPercent", "CurrentResidenceDuration", "LoanDuration", "Age", "ExistingCreditsCount", \
                                         "Dependents"], outputCol="features")

In [18]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_CheckingStatus, si_CreditHistory, si_EmploymentDuration, si_ExistingSavings, si_ForeignWorker, si_Housing, si_InstallmentPlans, si_Job, si_LoanPurpose, si_OthersOnLoan,\
                               si_OwnsProperty, si_Sex, si_Telephone, si_Label, va_features, classifier, label_converter])
model = pipeline.fit(train_data)

In [19]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderROC')
area_under_curve = evaluatorDT.evaluate(predictions)

evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderPR')
area_under_PR = evaluatorDT.evaluate(predictions)
#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve, "areaUnderPR = %g" % area_under_PR)

areaUnderROC = 0.708168 areaUnderPR = 0.632147


### 2.4 evaluate more metrics by exporting them into pandas and numpy

In [20]:
from sklearn.metrics import classification_report
y_pred = predictions.toPandas()['prediction']
y_pred = ['Risk' if pred == 1.0 else 'No Risk' for pred in y_pred]
y_test = test_data.toPandas()['Risk']
print(classification_report(y_test, y_pred, target_names=['Risk', 'No Risk']))

              precision    recall  f1-score   support

        Risk       0.79      0.91      0.84       657
     No Risk       0.73      0.51      0.60       327

   micro avg       0.77      0.77      0.77       984
   macro avg       0.76      0.71      0.72       984
weighted avg       0.77      0.77      0.76       984



## 2.5 Publish the model

In this section, the notebook uses Watson Machine Learning to save the model (including the pipeline) to the WML instance. Previous versions of the model are removed so that the notebook can be run again, resetting all data for another demo.

In [21]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### 2.5.1 Set default space

This is a new feature in CP4D, in order to deploy a model, you would have to create different
 deployment spaces and deploy your models there. You can list all the spaces using the .list()
 function, or you can create new spaces by going to CP4D menu on top left corner --> analyze -->
 analytics deployments --> New Deployment Space. Once you know which space you want to deploy
 in, simply use the GUID of the space as argument for .set.default_space() function below


In [22]:
wml_client.spaces.list()

------------------------------------  ----------------------  ------------------------
GUID                                  NAME                    CREATED
346b75fd-018d-4465-8cb8-0985406cfdee  Drift deployment space  2020-06-15T15:56:42.905Z
------------------------------------  ----------------------  ------------------------


We'll use the `GUID` for your Deployment space as listed for  the `default_space` in the method below:

In [23]:
wml_client.set.default_space('346b75fd-018d-4465-8cb8-0985406cfdee')

'SUCCESS'

Alternately, set `space_name` below and use the following cell to create a space with that name

In [24]:
# space_name = "my_space_name"
# spaces = wml_client.spaces.get_details()['resources']
# space_id = None
# for space in spaces:
#     if space['entity']['name'] == space_name:
#         space_id = space["metadata"]["guid"]
# if space_id is None:
#    space_id = wml_client.spaces.store(
#        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]
#wml_client.set.default_space(space_id)

### 2.5.2 Remove existing model and deployment

In [25]:
deployment_details = wml_client.deployments.get_details()
for deployment in deployment_details['resources']:
    deployment_id = deployment['metadata']['guid']
    model_id = deployment['entity']['asset']['href'].split('/')[3].split('?')[0]
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()


Deleting deployment id 0e2cf5ef-f002-4e3c-9a57-f32c82d2d232
Deleting model id 4ff783d3-6689-4ee2-b38c-3ebc245ab025
------------------------------------  --------------------  ------------------------  ---------
GUID                                  NAME                  CREATED                   TYPE
f4c467da-bc5b-4953-8518-968f3b8a0518  SAMAYA-06-15-2_MODEL  2020-06-16T03:16:33.002Z  mllib_2.3
3f603c63-2891-40df-ae66-8df0e81c90c5  SAMAYA-06-15_MODEL    2020-06-15T22:49:17.002Z  mllib_2.3
4414335f-fd76-4369-9ee7-caade5956459  SAMAYA-06-14_MODEL    2020-06-15T20:23:31.002Z  mllib_2.3
------------------------------------  --------------------  ------------------------  ---------


### 2.5.3 Set `training_data_reference`

In [26]:
training_data_reference = {                                                             
    "name": "Credit Risk feedback",
    "connection": DATABASE_CREDENTIALS,
    "source": {
        "tablename": "CREDIT_RISK_TRAINING",
        'schema_name': 'TRAININGDATA',
        "type": "db2"
    }
}

### 2.5.4 Store the model in Watson Machine Learning on CP4D

In [27]:
wml_models = wml_client.repository.get_model_details()
model_uid = None

for model_in in wml_models['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")
    metadata = {
        wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
        wml_client.repository.ModelMetaNames.TYPE: 'mllib_2.3',
        wml_client.repository.ModelMetaNames.RUNTIME_UID: 'spark-mllib_2.3',
    }

    published_model_details = wml_client.repository.store_model(model, metadata, training_data=df_data,  pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

Storing model ...
Done


In [28]:
model_uid

'336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775'

## 2.6 Deploy the model

The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [29]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")
    meta_props = {
        wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    deployment = wml_client.deployments.create(artifact_uid=model_uid, meta_props=meta_props)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

Deploying model...


#######################################################################################

Synchronous deployment creation for uid: '336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775' started

#######################################################################################


initializing.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4f95c2a9-5036-41b3-bc09-7b23900d4e70'
------------------------------------------------------------------------------------------------


Model id: 336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775
Deployment id: 4f95c2a9-5036-41b3-bc09-7b23900d4e70


# 3.0 Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [30]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [31]:
ai_client = APIClient4ICP(WOS_CREDENTIALS)
ai_client.version

'2.1.21'

## 3.1 Create datamart

### 3.1.1 Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If an OpenScale datamart exists in Db2, the existing datamart will be used and no data will be overwritten.

Prior instances of the Credit model will be removed from OpenScale monitoring.

In [32]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    print('Using existing external datamart')
except:
    print('Setting up external datamart')
    ai_client.data_mart.setup(db_credentials=DATABASE_CREDENTIALS, schema=SCHEMA_NAME)
    data_mart_details = ai_client.data_mart.get_details()

Using existing external datamart


## 3.2  Bind machine learning engines

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If this binding already exists, this code will output a warning message and use the existing binding.

In [33]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance4ICP(wml_credentials=WML_CREDENTIALS))
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = ai_client.data_mart.bindings.get_details()

In [34]:
binding_uid

'507f0e58-5ac1-4f22-bf92-fe51747b1d18'

In [35]:
ai_client.data_mart.bindings.list()

507f0e58-5ac1-4f22-bf92-fe51747b1d18,WML instance,watson_machine_learning,2020-06-16T05:32:13.707Z
d45d24c4-113a-470d-9669-2a39ed49faf5,WML instance,watson_machine_learning,2020-06-16T04:34:12.349Z
69eeed33-8d5b-4c4c-950b-97dd7f1c4cca,WML instance,watson_machine_learning,2020-06-16T03:16:56.422Z
3ee275c9-1325-488b-93ec-23a07ad5d54a,WML instance,watson_machine_learning,2020-06-15T22:49:40.887Z
fb3a6090-36eb-412a-a465-6b4ecd1a3bd4,WML instance,watson_machine_learning,2020-06-15T20:30:51.235Z
8c569603-7483-46eb-b722-1652ea5a6d36,WML instance,watson_machine_learning,2020-06-15T16:50:04.080Z
405e817b-3715-453b-91a7-6a36186de087,WML instance,watson_machine_learning,2020-06-15T16:26:24.549Z


## 3.3 Subscriptions

In [36]:
ai_client.data_mart.bindings.list_assets()

336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775,SAMAYA-DRIFT_MODEL,2020-06-16T05:31:52.002Z,model,mllib_2.3,405e817b-3715-453b-91a7-6a36186de087,False
336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775,SAMAYA-DRIFT_MODEL,2020-06-16T05:31:52.002Z,model,mllib_2.3,8c569603-7483-46eb-b722-1652ea5a6d36,False
336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775,SAMAYA-DRIFT_MODEL,2020-06-16T05:31:52.002Z,model,mllib_2.3,fb3a6090-36eb-412a-a465-6b4ecd1a3bd4,False
336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775,SAMAYA-DRIFT_MODEL,2020-06-16T05:31:52.002Z,model,mllib_2.3,3ee275c9-1325-488b-93ec-23a07ad5d54a,False
336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775,SAMAYA-DRIFT_MODEL,2020-06-16T05:31:52.002Z,model,mllib_2.3,69eeed33-8d5b-4c4c-950b-97dd7f1c4cca,False
336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775,SAMAYA-DRIFT_MODEL,2020-06-16T05:31:52.002Z,model,mllib_2.3,d45d24c4-113a-470d-9669-2a39ed49faf5,False
336d9ef5-aa6e-42b1-bf8a-c41dbb0d9775,SAMAYA-DRIFT_MODEL,2020-06-16T05:31:52.002Z,model,mllib_2.3,507f0e58-5ac1-4f22-bf92-fe51747b1d18,False
f4c467da-bc5b-4953-8518-968f3b8a0518,SAMAYA-06-15-2_MODEL,2020-06-16T03:16:33.002Z,model,mllib_2.3,405e817b-3715-453b-91a7-6a36186de087,False
f4c467da-bc5b-4953-8518-968f3b8a0518,SAMAYA-06-15-2_MODEL,2020-06-16T03:16:33.002Z,model,mllib_2.3,8c569603-7483-46eb-b722-1652ea5a6d36,False
f4c467da-bc5b-4953-8518-968f3b8a0518,SAMAYA-06-15-2_MODEL,2020-06-16T03:16:33.002Z,model,mllib_2.3,fb3a6090-36eb-412a-a465-6b4ecd1a3bd4,False
f4c467da-bc5b-4953-8518-968f3b8a0518,SAMAYA-06-15-2_MODEL,2020-06-16T03:16:33.002Z,model,mllib_2.3,3ee275c9-1325-488b-93ec-23a07ad5d54a,False


In [ ]:
ai_client.data_mart.bindings.get_details(binding_uid)

### 3.3.1 Remove existing credit risk subscriptions

This code removes previous subscriptions to the Credit model to refresh the monitors with the new model and new data.

In [38]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [39]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(        
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='Risk',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Get subscription list

In [40]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

c0feeb64-26d5-436d-806a-4954c3235bf0,SAMAYA-DRIFT_MODEL,model,405e817b-3715-453b-91a7-6a36186de087,2020-06-16T05:32:46.827Z


In [41]:
subscription_details = subscription.get_details()

# 4.0 Generate drift model <a name="driftmodel"></a>


Drift requires a trained model to be uploaded manually for WML. You can train, create and download a drift detection model using the code below. The entire code can be found [here](https://github.com/IBM-Watson/aios-data-distribution/blob/master/training_statistics_notebook.ipynb) ( check for Drift detection model generation). 

In [42]:
training_data_info = {
  "class_label":'Risk',
   "feature_columns":["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    "categorical_columns":["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
}


In [43]:
#Set model_type. Acceptable values are:["binary","multiclass","regression"]
model_type = "binary"
#model_type = "multiclass"
#model_type = "regression"

In [44]:
def score(training_data_frame):
     #To be filled by the user
      WML_CREDENTAILS = WML_CREDENTIALS
      
      
      #The data type of the label column and prediction column should be same .
      #User needs to make sure that label column and prediction column array should have the same unique class labels
      prediction_column_name = "predictedLabel"
      probability_column_name = "probability"
        
      feature_columns = list(training_data_frame.columns)
      training_data_rows = training_data_frame[feature_columns].values.tolist()
      #print(training_data_rows)
    

    
      payload_scoring = {
          wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
               "fields": feature_columns,
               "values": [x for x in training_data_rows]
          }]
      }
      
      score = wml_client.deployments.score(deployment_uid, payload_scoring)
      score_predictions = score.get('predictions')[0]
      
      prob_col_index = list(score_predictions.get('fields')).index(probability_column_name)
      predict_col_index = list(score_predictions.get('fields')).index(prediction_column_name)
      
      if prob_col_index < 0 or predict_col_index < 0:
          raise Exception("Missing prediction/probability column in the scoring response")
          
      import numpy as np
      probability_array = np.array([value[prob_col_index] for value in score_predictions.get('values')])
      prediction_vector = np.array([value[predict_col_index] for value in score_predictions.get('values')])
      
      return probability_array, prediction_vector

In [45]:
#Generate drift detection model
from ibm_wos_utils.drift.drift_trainer import DriftTrainer

drift_detection_input = {
        "feature_columns":training_data_info.get('feature_columns'),
        "categorical_columns":training_data_info.get('categorical_columns'),
        "label_column": training_data_info.get('class_label'),
        "problem_type": model_type
    }
    
    
drift_trainer = DriftTrainer(data_df,drift_detection_input)
if model_type != "regression":
        #Note: batch_size can be customized by user as per the training data size
    drift_trainer.generate_drift_detection_model(score,batch_size=data_df.shape[0])
    
    #Note: Two column constraints are not computed beyond two_column_learner_limit(default set to 200)
    #User can adjust the value depending on the requirement
drift_trainer.learn_constraints(two_column_learner_limit=200)
drift_trainer.create_archive()

Learning two feature constraints...: 100%|██████████| 209/209 [00:03<00:00, 68.65constraints/s] 


In [46]:
#Generate a download link for drift detection model
from IPython.display import HTML
import base64
import io

def create_download_link_for_ddm( title = "Download Drift detection model", filename = "drift_detection_model.tar.gz"):  
    
    #Retains stats information    

    with open(filename,'rb') as file:
        ddm = file.read()
    b64 = base64.b64encode(ddm)
    payload = b64.decode()
        
    html = '<a download="{filename}" href="data:text/json;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
    
create_download_link_for_ddm()

In [47]:
#!rm -rf drift_detection_model.tar.gz
#!wget -O drift_detection_model.tar.gz https://github.com/IBM/cpd-intelligent-loan-agent-assets/blob/master/models/drift_detection_model.tar.gz?raw=true

# 5.0 Submit payload <a name="payload"></a>

### Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. 

In [48]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(deployment_uid, payload)



print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

Single record scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, [21, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19, 20], [1.0, 1.0, 1.0, 13

# 6. Enable drift monitoring <a name="monitor"></a>

In [49]:
subscription.drift_monitoring.enable(threshold=0.05, min_records=10,model_path="drift_detection_model.tar.gz")

{'config_status': {'state': 'finished'},
 'data_drift_enabled': True,
 'drift_threshold': 0.05,
 'is_schedule_enabled': True,
 'min_samples': 10,
 'model_drift_enabled': True,
 'next_scheduled_run_timestamp': '2020-06-16T05:35:54.841069Z',
 'schedule_repeat_interval': 3,
 'schedule_repeat_type': 'hour'}

# 7. Run Drift monitor on demand <a name="driftrun"></a>

In [50]:
!rm german_credit_feed.json
!wget https://raw.githubusercontent.com/IBM/cpd-intelligent-loan-agent-assets/master/data/german_credit_feed.json

--2020-06-16 05:33:58--  https://raw.githubusercontent.com/IBM/cpd-intelligent-loan-agent-assets/master/data/german_credit_feed.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3076548 (2.9M) [text/plain]
Saving to: ‘german_credit_feed.json’

100%[======================================>] 3,076,548   --.-K/s   in 0.06s   

2020-06-16 05:33:58 (52.5 MB/s) - ‘german_credit_feed.json’ saved [3076548/3076548]



In [58]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(10):
    current = random.choice(scoring_data['values'])
    #set age of all rows to 100 to increase drift values on dashboard
    current[12] = 100
   
    values.append(current)
payload_scoring = {"fields": fields, "values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(deployment_uid, payload)

In [59]:
drift_run_details = subscription.drift_monitoring.run(background_mode=False)




 Waiting for end of drift monitoring run  




RUNNING
COMPLETED

---------------------------
 Successfully finished run 
---------------------------




In [ ]:
subscription.drift_monitoring.get_table_content()


## Congratulations!

You have finished running all the cells within the notebook for IBM Watson OpenScale. You can now view the OpenScale dashboard by going to the CP4D `Home` page, and clicking `Services`. Choose the `OpenScale` tile and click the menu to `Open`. Click on the tile for the model you've created to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and can be deleted if necessary.